## Course Project

In [5]:
import pandas as pd
from apyori import apriori, load_transactions
from datetime import datetime, date, time, timedelta


In [28]:
df = pd.read_csv('crime.csv')

## Data Preprocessing
### Data Cleaning


In [29]:
# Only focus on IS_CRIME
df = df.loc[df['IS_CRIME'] == True]

### Data Reduction

In [30]:
# For  Apriori: OFFENSE_CATEGORY_ID, NEIGHBORHOOD_ID
# df = df[["OFFENSE_CATEGORY_ID", "NEIGHBORHOOD_ID" ]]

## Frequent Patterns
###  Apriori Algorithm 

In [31]:
association_rules = apriori(df.values[0:10],min_support = 0.02, min_confidence = 0.1, min_lift = 3)
association_results = list(association_rules)

TypeError: '<' not supported between instances of 'str' and 'int'

In [32]:
results = []
for item in association_results:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    
    value0 = str(items[0])
    value1 = str(items[1])

    #second index of the inner list
    value2 = str(item[1])[:7]

    #third index of the list located at 0th
    #of the third index of the inner list

    value3 = str(item[2][0][2])[:7]
    value4 = str(item[2][0][3])[:7]
    
    rows = (value0, value1,value2,value3,value4)
    results.append(rows)
    
labels = ['Crime','Neighborhood','Support','Confidence','Lift']
pretty_print = pd.DataFrame.from_records(results, columns = labels)

print(pretty_print)

                      Crime                Neighborhood Support Confidence  \
0            city-park-west            all-other-crimes     0.1    0.33333   
1                 montbello            all-other-crimes     0.1    0.33333   
2                     speer            all-other-crimes     0.1    0.33333   
3                   belcaro                     larceny     0.1        1.0   
4              cherry-creek                     larceny     0.1        1.0   
5              drug-alcohol                   goldsmith     0.1        1.0   
6                   larceny  gateway-green-valley-ranch     0.1        1.0   
7  theft-from-motor-vehicle                    highland     0.1        1.0   
8                villa-park    theft-from-motor-vehicle     0.1    0.33333   
9                 wellshire    theft-from-motor-vehicle     0.1    0.33333   

      Lift  
0  3.33333  
1  3.33333  
2  3.33333  
3  3.33333  
4  3.33333  
5     10.0  
6  3.33333  
7  3.33333  
8  3.33333  
9  3.33333 

### Apriori Algorithm Version 2
https://travis-ci.org/asaini/Apriori.svg?branch=master

#### Data Reduction

In [35]:
df2 = df[["OFFENSE_CATEGORY_ID", "NEIGHBORHOOD_ID","FIRST_OCCURRENCE_DATE"]]
# df2 = df2[df2.OFFENSE_CATEGORY_ID != "all-other-crimes"]

df2['DATE'], df2['TIME'] = df2['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str
df2['DATE'] = pd.to_datetime(df2['DATE'])
df2['DAY_OF_WEEK'] = df2['DATE'].dt.day_name()
df2

/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,OFFENSE_CATEGORY_ID,NEIGHBORHOOD_ID,FIRST_OCCURRENCE_DATE,DATE,TIME,DAY_OF_WEEK
0,all-other-crimes,montbello,6/15/2016 11:31:00 PM,2016-06-15,11:31:00 PM,Wednesday
1,larceny,gateway-green-valley-ranch,10/11/2017 12:30:00 PM,2017-10-11,12:30:00 PM,Wednesday
2,theft-from-motor-vehicle,wellshire,3/4/2016 8:00:00 PM,2016-03-04,8:00:00 PM,Friday
3,larceny,belcaro,1/30/2018 7:20:00 PM,2018-01-30,7:20:00 PM,Tuesday
4,larceny,cherry-creek,6/22/2017 8:53:00 PM,2017-06-22,8:53:00 PM,Thursday
...,...,...,...,...,...,...
528579,theft-from-motor-vehicle,union-station,10/27/2019 8:00:00 PM,2019-10-27,8:00:00 PM,Sunday
528581,robbery,east-colfax,10/27/2019 11:45:00 PM,2019-10-27,11:45:00 PM,Sunday
528583,other-crimes-against-persons,dia,10/28/2019 1:13:00 AM,2019-10-28,1:13:00 AM,Monday
528584,other-crimes-against-persons,sunnyside,10/28/2019 2:10:00 AM,2019-10-28,2:10:00 AM,Monday


## Split by days

In [108]:
daysOfWeek = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
days = []
i = 0

for day in daysOfWeek:
    x = df2[['OFFENSE_CATEGORY_ID', 'TIME', 'NEIGHBORHOOD_ID']]
    days.append(x)
    days[i] = days[i].loc[df2.DAY_OF_WEEK == day]
    days[i]['TIME'] = pd.to_datetime(days[i]['TIME'], format='%I:%M:%S %p')
    i += 1
    
days[0]

/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,OFFENSE_CATEGORY_ID,TIME,NEIGHBORHOOD_ID
11,theft-from-motor-vehicle,1900-01-01 05:25:00,ruby-hill
15,theft-from-motor-vehicle,1900-01-01 02:00:00,highland
29,all-other-crimes,1900-01-01 20:00:00,stapleton
36,auto-theft,1900-01-01 22:00:00,rosedale
50,public-disorder,1900-01-01 06:30:00,cherry-creek
...,...,...,...
528576,public-disorder,1900-01-01 19:45:00,highland
528577,robbery,1900-01-01 21:51:00,gateway-green-valley-ranch
528578,auto-theft,1900-01-01 10:00:00,bear-valley
528579,theft-from-motor-vehicle,1900-01-01 20:00:00,union-station


In [107]:
block1 = '1900-01-01 04:00:00'
block2 = '1900-01-01 08:00:00'
block3 = '1900-01-01 12:00:00'
block4 = '1900-01-01 16:00:00'
block5 = '1900-01-01 20:00:00'
block6 = '1900-01-01 23:59:00'
datetime_object1 = datetime.strptime(block1, '%Y-%m-%d %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%Y-%m-%d %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%Y-%m-%d %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%Y-%m-%d %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%Y-%m-%d %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%Y-%m-%d %H:%M:%S')

In [109]:
for i in range(len(days)):
    days[i].loc[days[i]['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

    days[i].loc[(days[i]['TIME'].dt.time > datetime_object1.time())&(days[i]['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

    days[i].loc[(days[i]['TIME'].dt.time > datetime_object2.time())&(days[i]['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

    days[i].loc[(days[i]['TIME'].dt.time > datetime_object3.time())&(days[i]['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

    days[i].loc[(days[i]['TIME'].dt.time > datetime_object4.time())&(days[i]['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

    days[i].loc[(days[i]['TIME'].dt.time > datetime_object5.time())&(days[i]['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6

    days[i]['TIME'] = [d.time() for d in days[i]['TIME']]

days[0]

/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,OFFENSE_CATEGORY_ID,TIME,NEIGHBORHOOD_ID
11,theft-from-motor-vehicle,08:00:00,ruby-hill
15,theft-from-motor-vehicle,04:00:00,highland
29,all-other-crimes,20:00:00,stapleton
36,auto-theft,23:59:00,rosedale
50,public-disorder,08:00:00,cherry-creek
...,...,...,...
528576,public-disorder,20:00:00,highland
528577,robbery,23:59:00,gateway-green-valley-ranch
528578,auto-theft,12:00:00,bear-valley
528579,theft-from-motor-vehicle,20:00:00,union-station


In [111]:
WCC_csv = days[0][['OFFENSE_CATEGORY_ID', 'TIME', 'NEIGHBORHOOD_ID']]
WCC_csv = WCC_csv.sample(5000)
WCC_csv.to_csv('Sun.csv')

for i in range(len(days)):
    WCC_csv = days[i][['OFFENSE_CATEGORY_ID', 'TIME', 'NEIGHBORHOOD_ID']]
    WCC_csv = WCC_csv.sample(5000)
    WCC_csv.to_csv('Day' + i + '.csv')

!python apriori.py -f 'Day4.csv' -s 0.0012

TypeError: can only concatenate str (not "int") to str

## White Collar Crime

In [64]:
block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')

In [65]:
df_WCC = df2[['OFFENSE_CATEGORY_ID', 'TIME', 'DAY_OF_WEEK','NEIGHBORHOOD_ID']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime']
df_WCC_am =  df_WCC.loc[df_WCC.TIME.str[-2:] == "AM" ]
df_WCC_am['TIME'] = [x[:-3] for x in df_WCC_am['TIME']]
df_WCC_am['TIME'] =  pd.to_datetime(df_WCC_am['TIME'], format='%H:%M:%S')

df_WCC_am

/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,OFFENSE_CATEGORY_ID,TIME,DAY_OF_WEEK,NEIGHBORHOOD_ID
629,white-collar-crime,1900-01-01 01:00:00,Tuesday,dia
712,white-collar-crime,1900-01-01 12:00:00,Thursday,capitol-hill
730,white-collar-crime,1900-01-01 08:50:00,Thursday,five-points
814,white-collar-crime,1900-01-01 12:00:00,Wednesday,harvey-park
822,white-collar-crime,1900-01-01 12:00:00,Sunday,chaffee-park
...,...,...,...,...
527765,white-collar-crime,1900-01-01 08:55:00,Wednesday,speer
527821,white-collar-crime,1900-01-01 12:00:00,Tuesday,five-points
527998,white-collar-crime,1900-01-01 12:00:00,Wednesday,congress-park
528073,white-collar-crime,1900-01-01 12:00:00,Sunday,hampden


In [66]:
df_WCC = df_WCC[['OFFENSE_CATEGORY_ID', 'TIME', 'DAY_OF_WEEK','NEIGHBORHOOD_ID']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime']
df_WCC_pm =  df_WCC.loc[df_WCC.TIME.str[-2:] == "PM" ]
df_WCC_pm['TIME'] = [x[:-3] for x in df_WCC_pm['TIME']]
df_WCC_pm['TIME'] =  pd.to_datetime(df_WCC_pm['TIME'], format='%H:%M:%S')
df_WCC_pm['TIME'] = df_WCC_pm['TIME'] + timedelta(hours=12)
df_WCC_pm

/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/danielscott/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,OFFENSE_CATEGORY_ID,TIME,DAY_OF_WEEK,NEIGHBORHOOD_ID
18,white-collar-crime,1900-01-02 00:00:00,Thursday,washington-park-west
25,white-collar-crime,1900-01-02 00:00:00,Monday,hampden-south
248,white-collar-crime,1900-01-02 00:00:00,Tuesday,capitol-hill
289,white-collar-crime,1900-01-01 14:00:00,Wednesday,stapleton
366,white-collar-crime,1900-01-01 18:00:00,Monday,westwood
...,...,...,...,...
528002,white-collar-crime,1900-01-01 13:00:00,Thursday,north-capitol-hill
528020,white-collar-crime,1900-01-01 14:07:00,Thursday,lincoln-park
528215,white-collar-crime,1900-01-01 17:00:00,Monday,dia
528224,white-collar-crime,1900-01-01 13:15:00,Thursday,lincoln-park


In [67]:
final_WCC = pd.concat([df_WCC_am, df_WCC_pm], ignore_index=True, sort =False)

final_WCC.loc[final_WCC['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object1.time())&(final_WCC['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object2.time())&(final_WCC['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object3.time())&(final_WCC['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object4.time())&(final_WCC['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object5.time())&(final_WCC['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6


final_WCC['TIME'] = [d.time() for d in final_WCC['TIME']]
final_WCC

,OFFENSE_CATEGORY_ID,TIME,DAY_OF_WEEK,NEIGHBORHOOD_ID
0,white-collar-crime,04:00:00,Tuesday,dia
1,white-collar-crime,12:00:00,Thursday,capitol-hill
2,white-collar-crime,12:00:00,Thursday,five-points
3,white-collar-crime,12:00:00,Wednesday,harvey-park
4,white-collar-crime,12:00:00,Sunday,chaffee-park
...,...,...,...,...
7233,white-collar-crime,16:00:00,Thursday,north-capitol-hill
7234,white-collar-crime,16:00:00,Thursday,lincoln-park
7235,white-collar-crime,20:00:00,Monday,dia
7236,white-collar-crime,16:00:00,Thursday,lincoln-park


In [71]:
final_WCC[['DAY_OF_WEEK','NEIGHBORHOOD_ID', 'TIME']]

,DAY_OF_WEEK,NEIGHBORHOOD_ID,TIME
0,Tuesday,dia,04:00:00
1,Thursday,capitol-hill,12:00:00
2,Thursday,five-points,12:00:00
3,Wednesday,harvey-park,12:00:00
4,Sunday,chaffee-park,12:00:00
...,...,...,...
7233,Thursday,north-capitol-hill,16:00:00
7234,Thursday,lincoln-park,16:00:00
7235,Monday,dia,20:00:00
7236,Thursday,lincoln-park,16:00:00


In [70]:
WCC_csv = final_WCC[['DAY_OF_WEEK','NEIGHBORHOOD_ID', 'TIME']]
WCC_csv = WCC_csv.sample(5000)
WCC_csv.to_csv('WCC.csv')


!python apriori.py -f 'WCC.csv' -s 0.0012

item: ('Wednesday', 'southmoor-park') , 0.001
item: ('04:00:00', 'jefferson-park') , 0.001
item: ('west-highland', 'Wednesday') , 0.001
item: ('Tuesday', 'lincoln-park') , 0.001
item: ('west-highland', 'Tuesday') , 0.001
item: ('Wednesday', 'college-view-south-platte') , 0.001
item: ('platt-park', '16:00:00') , 0.001
item: ('overland', 'Monday') , 0.001
item: ('20:00:00', 'civic-center') , 0.001
item: ('Saturday', 'windsor') , 0.001
item: ('04:00:00', 'villa-park') , 0.001
item: ('villa-park', '16:00:00') , 0.001
item: ('elyria-swansea', 'Tuesday') , 0.001
item: ('Thursday', 'barnum') , 0.001
item: ('clayton', '16:00:00') , 0.001
item: ('auraria', 'Tuesday') , 0.001
item: ('elyria-swansea', 'Wednesday') , 0.001
item: ('bear-valley', '20:00:00') , 0.001
item: ('cory-merrill', 'Friday') , 0.001
item: ('20:00:00', 'south-park-hill') , 0.001
item: ('cheesman-park', 'Tuesday') , 0.001
item: ('sun-valley', '12:00:00') , 0.001
item: ('clayton', '08:00:00') , 0.001
item: ('Saturday', 'college-

***

In [69]:
test2 = df2[['DAY_OF_WEEK', 'NEIGHBORHOOD_ID','OFFENSE_CATEGORY_ID']]
test2 = test2.sample(1000)
test2.to_csv('test2.csv')

In [22]:
!python apriori.py -f 'test2.csv' -s 0.0012

item: ('Monday', 'regis') , 0.002
item: ('Thursday', 'university-park') , 0.002
item: ('Sunday', 'capitol-hill') , 0.002
item: ('sloan-lake', 'Friday') , 0.002
item: ('university-hills', 'Sunday') , 0.002
item: ('public-disorder', 'union-station') , 0.002
item: ('auto-theft', 'gateway-green-valley-ranch') , 0.002
item: ('cherry-creek', 'Wednesday') , 0.002
item: ('east-colfax', 'Sunday') , 0.002
item: ('cbd', 'aggravated-assault') , 0.002
item: ('overland', 'other-crimes-against-persons') , 0.002
item: ('Monday', 'north-capitol-hill') , 0.002
item: ('highland', 'Sunday') , 0.002
item: ('drug-alcohol', 'south-park-hill') , 0.002
item: ('university-park', 'larceny') , 0.002
item: ('northeast-park-hill', 'Tuesday') , 0.002
item: ('Sunday', 'west-colfax') , 0.002
item: ('robbery', 'capitol-hill') , 0.002
item: ('public-disorder', 'goldsmith') , 0.002
item: ('Monday', 'union-station') , 0.002
item: ('ruby-hill', 'Tuesday') , 0.002
item: ('Sunday', 'virginia-village') , 0.002
item: ('Tuesday